<a href="https://colab.research.google.com/github/simecek/PseudoDNA_Generator/blob/master/models/Exons_WholeGenomeButChr1_v1_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup

In [1]:
!pip install fastai2>=0.0.11 ipywidgets matplotlib nbdev>=0.2.12 pandas scikit_learn sentencepiece

In [2]:
from fastai2.text.all import *
import pandas as pd
import torch

In [3]:
torch.cuda.is_available(), torch.cuda.device_count(), torch.cuda.get_device_name(0)

(True, 1, 'Tesla P100-PCIE-16GB')

In [4]:
# Mount to your Google Drive allowing lesson files will be saved to your Drive location
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [5]:
!wget https://raw.githubusercontent.com/simecek/PseudoDNA_Generator/master/models/genomic_tokenizer2.py

--2020-06-27 18:29:12--  https://raw.githubusercontent.com/simecek/PseudoDNA_Generator/master/models/genomic_tokenizer2.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 565 [text/plain]
Saving to: ‘genomic_tokenizer2.py’

genomic_tokenizer2. 100%[===================>]     565  --.-KB/s    in 0s      

2020-06-27 18:29:12 (30.1 MB/s) - ‘genomic_tokenizer2.py’ saved [565/565]



In [6]:
from genomic_tokenizer2 import tkn2

In [7]:
dt = pd.read_csv("/content/drive/My Drive/data/random/random_exons.csv")
dt.columns = ['chr', 'start', 'end', 'seq']
dt = dt[~dt.seq.str.contains("N")]   # just for sure

train = dt[dt.chr!="1"]
test = dt[dt.chr=="1"]

print(dt.shape, train.shape, test.shape)
dt.head()

(50000, 4) (45336, 4) (4664, 4)


,chr,start,end,seq
0,1,236763182,236763381,GACTACCCAAAAGATTTTTGGCACTTACAATTTTTAAAATAGTTTATGTCATCTCTTCATTATTTAGGGCTGGATGGTCAACTCAGTCAGTGATTTTTTGATGCTTCTCTTATCCTCCAGAATAGAGACCTAAGGACACGTGGAAGTCAGTTTAATTGCCAGAGAGAAGGATGCAATCACTAGGTAAAATGAGGTTTTTA
1,17,8289582,8289781,AGCCCCCGTAAGGAGGAAGGAACGGGCGGGTATCTCATGACTGGGTTCCCAGGAGAATCGGGCTGGGAGGGACAGAACAGGGAGACTCACTGGTGGGATCCTCCAAGGAAGCAGGAGTGGGCCAGAGGTTTGGGGTAACTGATACCCAGGTCCTCTGGGGAACAGGAATTGGCAATTTCCAAACCTCAGGCCTGGATGAT
2,3,100455979,100456178,TGAGCCCCAATTCACCATTTCAGGATGTGGATGGGGGCGGGGTTGGGGGTAAAAACAGCTATAAAAAGCAACTGCAGATGCTGACTGACTGCAGTGGGCAGGGTATGTAGCTGCTCCAAGATGACTTGCATCATACCCCAATTACTGCTGGCATCTTAGTTGAGAGTATAATCTGCTTGGTTGCCTTTTTATGGGAATAA
3,10,94852754,94852953,AGGGCCTGGCCCGCATGGAGCTGTTTTTATTCCTGACCTTCATTTTACAGAACTTTAACCTGAAATCTCTGATTGACCCAAAGGACCTTGACACAACTCCTGTTGTCAATGGATTTGCTTCTGTCCCGCCCTTCTATCAGCTGTGCTTCATTCCTGTCTGAAGAAGCACAGATGGTCTGGCTGCTCCTGTGCTGTCCCTG
4,1,209756085,209756284,GGGTTTGCGAACCTTGTGGCTTTGTCTGTTTCCTGTTTCAGCAAGGCTGCTGTGCAAAAAGAAAGACGGTACCAAGAAGGGACGTGTTAATGGGGCCCAGACCTATGGATTGAAAGCGTGTGCTTTACCCATCTGCTGTCTTGCTCCATCTGAGACCAGAGCCAAGATCTGCCCAGGACTGGAATGCTTTCCCGAGTGGC


In [8]:
!rm -rf split_tok

!rm -rf split
!mkdir split
!mkdir split/train
!mkdir split/valid
!mkdir split/train/1/
!mkdir split/valid/1/

In [9]:
# splitting the file into training and test part
N = len(dt.seq)

for i, s in enumerate(train.seq):
  open("split/train/1/seq"+str(i)+".txt", 'w').writelines([s])
for i, s in enumerate(test.seq):
  open("split/valid/1/seq"+str(i)+".txt", 'w').writelines([s])

## Data Loaders

In [10]:
dls_lm = TextDataLoaders.from_folder(Path("./split"), bs=128, seed=42, 
                                   is_lm=True, 
                                   tok_tfm=tkn2, seq_len=50)

In [11]:
dls_lm.show_batch()

,text,text_
0,G C C T T T C G T A C T C A C C T T G G C A G C A C T G A A A C T C A A G G A G C A A A G C T T C T,C C T T T C G T A C T C A C C T T G G C A G C A C T G A A A C T C A A G G A G C A A A G C T T C T C
1,C A G C A A C C A T T G G T G C C G A G G A C C A T G G G T C C A T G C A A A A C G A C A T G G G C,A G C A A C C A T T G G T G C C G A G G A C C A T G G G T C C A T G C A A A A C G A C A T G G G C G
2,C C A C T G C A C T C C A G C C T G G G C G A C A G A G C A A G A C C C T G T C T C A A A A A C A A,C A C T G C A C T C C A G C C T G G G C G A C A G A G C A A G A C C C T G T C T C A A A A A C A A A
3,C A A T C C C T G A G G A C T G A T T T C A G A G T G A C T A C A C A C A G T A C G A A C C T A C A,A A T C C C T G A G G A C T G A T T T C A G A G T G A C T A C A C A C A G T A C G A A C C T A C A G
4,A A G G C T T G A G C C C A G T A G T T C G G G G C T G C A G T G A G C C A T G A T T G T G C C A C,A G G C T T G A G C C C A G T A G T T C G G G G C T G C A G T G A G C C A T G A T T G T G C C A C T
5,C T A C C C C C A G A G G A T C A G A C T G C T G A C G T G C T T C C C G T T A C C A G G G A A A C,T A C C C C C A G A G G A T C A G A C T G C T G A C G T G C T T C C C G T T A C C A G G G A A A C C
6,G G G A G T A G G C T T T G A G A A G T T G C T A C T C C A G G T G T G A A C C A C C A G G C G G T,G G A G T A G G C T T T G A G A A G T T G C T A C T C C A G G T G T G A A C C A C C A G G C G G T G
7,G A T C C T G T G A A G A A C C G C T G T G G A G G G C A G C A G C T G G A C G A G G G C A C A G G,A T C C T G T G A A G A A C C G C T G T G G A G G G C A G C A G C T G G A C G A G G G C A C A G G A
8,C G G C C G C T G T C C C C C C C C A C C A C T G C C G G G G C C C T G C C A A C C A C A C T G A G,G G C C G C T G T C C C C C C C C A C C A C T G C C G G G G C C C T G C C A A C C A C A C T G A G G


## Model and Learning

In [12]:
learn = language_model_learner(
    dls_lm, AWD_LSTM, drop_mult=0.3, pretrained=False, 
    metrics=[accuracy, Perplexity()])

In [13]:
learn = learn.load('/content/drive/My Drive/DNAModels/Human_v0/all_but_chr1_v2')
learn.unfreeze()
learn.summary()

SequentialRNN (Input shape: ['128 x 50'])
Layer (type)         Output Shape         Param #    Trainable 
RNNDropout           128 x 50 x 400       0          False     
________________________________________________________________
RNNDropout           128 x 50 x 1152      0          False     
________________________________________________________________
RNNDropout           128 x 50 x 1152      0          False     
________________________________________________________________
Linear               128 x 50 x 8         3,208      True      
________________________________________________________________
RNNDropout           128 x 50 x 400       0          False     
________________________________________________________________

Total params: 3,208
Total trainable params: 3,208
Total non-trainable params: 0

Optimizer used: <function Adam at 0x7f8f8d31c268>
Loss function: FlattenedLoss of CrossEntropyLoss()

Model unfrozen

Callbacks:
  - TrainEvalCallback
  - Recorder
  -

In [14]:
learn.fit_one_cycle(10, 1e-2)
learn.export("/content/drive/My Drive/DNAModels/Exons/all_but_chr1_v2_finetuned.pkl")

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,1.442673,1.331834,0.338965,3.787983,07:07
1,1.327743,1.324584,0.349833,3.760620,06:55
2,1.324371,1.327430,0.351272,3.771337,06:57
3,1.323703,1.320198,0.354371,3.744163,07:08
4,1.321296,1.321004,0.356050,3.747181,07:03
5,1.319578,1.317745,0.357292,3.734990,07:00
6,1.316106,1.315158,0.359315,3.725340,07:03
7,1.312619,1.312589,0.361487,3.715782,07:06
8,1.311798,1.310938,0.362555,3.709652,07:06
9,1.310632,1.310519,0.362889,3.708097,07:05
